# Подготовка массивов данных для обучения ИНС

1. Загрузка полного массива данных
2. Формирование косинусовых векторов для ИНС директора
3. Формирование прецедентов для ИНС распознавателя

## Загрузка полного массива данных

In [6]:
_DIR_ = 'D:/DNN/cross5_rank1000/face2/'
LIMIT = 50000
DIM = 5
ETA = 20
PP_K = 2*ETA*(DIM - 1) + 1
print(PP_K)

161


In [7]:
import numpy as np

dataset = np.load(_DIR_ + 'dataset100000_face2.npy')
#dataset[:,0:PP_K] -> distances
#dataset[:,PP_K] -> face
#dataset[:,PP_K + 1:PP_K + 1 + DIM] -> cosine coordinates
#dataset[:,PP_K + 1 + DIM:PP_K + 1 + 2*DIM] -> face count
print(len(dataset))

20700


## Формирование косинусовых векторов для ИНС директора 1 (без нулей)

Поместим в `face1_yes` строчки, в которых поле `face` равно `1.`. Движение возможно только по одиночной гиперплоскости.

Сохраним в `test` поля, содержащие образ рецептивного поля, а в `answers` - косинусовые координаты.

Объединим `test` и `answers` в массив `face_vec`. Перемешаем строчки и сохраним в `face_vec.npy`.

## Формирование косинусовых векторов для ИНС директора 2 (без нулей)

Поместим в `face1_yes` строчки, в которых поле `face` равно `1.`. Движение возможно только по ребру.

In [8]:
face1_yes = dataset[np.in1d(dataset[:, PP_K], np.asarray([2.]))]
#face1_yes = face1_yes[np.in1d(face1_yes[:, PP_K + 1 + DIM + 1], np.asarray([1.]))]
print(face1_yes[:10, PP_K + 1 + DIM + 1]) # Проверка столбца face_count[1] (содержит число многообразий уровня 2, по которым возможно движение)
print(len(face1_yes))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
20700


Сохраним в `test` поля, содержащие образ рецептивного поля, а в `answers` - косинусовые координаты.

In [9]:
test = face1_yes[:,:PP_K]
answers = face1_yes[:,PP_K + 1:PP_K + 1 + DIM]
#answers[:,0] = 0.
print(answers[-5:])

[[ 7.56999543e-16  5.22232968e-01 -1.05940288e-01  8.46197012e-01
  -9.46249429e-17]
 [ 1.76615224e-16 -2.74873708e-01  1.34634177e-01 -6.02102344e-01
   7.37421758e-01]
 [-8.21517238e-15 -9.79503923e-02 -4.71988102e-01 -8.66769571e-01
   1.27841553e-01]
 [-8.28403439e-15 -4.23430354e-01 -8.86034988e-01 -1.04658924e-01
  -1.57147206e-01]
 [ 3.42041036e-16 -1.60128154e-01  4.93548117e-01 -5.86422546e-01
  -6.21995039e-01]]


Объединим `test` и `answers` в массив `face_vec`. Перемешаем строчки и сохраним в `face_vec.npy`.

In [10]:
face1_vec = np.hstack((test,answers))
np.random.shuffle(face1_vec)
print(face1_vec[:5,-5:])
np.save(_DIR_ + 'face2_vec.npy', face1_vec)

[[ 0.00000000e+00 -3.09492230e-01 -6.47006840e-01 -6.96847693e-01
   0.00000000e+00]
 [-4.12575345e-17 -2.03615741e-01 -9.40460821e-01  3.77425678e-02
   2.69535847e-01]
 [-3.80612244e-16 -3.55454959e-01 -5.98495803e-01  2.30751773e-01
  -6.79858930e-01]
 [ 2.86932111e-16 -5.56348640e-01 -5.76586073e-01 -5.98351645e-01
   0.00000000e+00]
 [-4.48255102e-16  5.04878164e-01 -5.72289806e-01 -6.01140083e-01
  -2.37092846e-01]]


## Формирование косинусовых векторов для ИНС директора (включая нули)

## Формирование прецедентов для ИНС распознавателя

Поместим в `face1_yes` строчки, в которых поле `face` равно `2.` Движение возможно только по ребрам. Причем, по единственному ребру.

Поместим в `face1_no` строчки, в которых поле `face` не равно `2.`.

Поместим в `idx` случайный набор индексов. По этим индексам выпишем в `test` случайный набор строк из `face1_yes`. Сгенерируем столбец правильных ответов, состоящий из единиц и поместим его в `answers`. Объединив `test` и `answers`, перезапишем в `face1_yes` набор данных, где первые PP_K столбцов содержат образ рецептивного поля (набор смещений), а последний столбец правильные ответы (единицы).

Поместим в `idx` случайный набор индексов. По этим индексам выпишем в `test` случайный набор строк из `face1_no`. Сгенерируем столбец правильных ответов, состоящий из нулей и поместим его в `answers`. Объединив `test` и `answers`, перезапишем в `face1_no` набор данных, где первые PP_K столбцов содержат образ рецептивного поля (набор смещений), а последний столбец правильные ответы (нули).

Объединим и поместим в `face1` наборы положительных прецедентов `face1_yes` и отрицательных прецедентов `face1_no`. Перемешаем их и сохраним в файл `face_def.npy`.